
<h1 align="center"> CSE 164 Assignment 3, Spring 2022



<h2 align="center"> 5 Questions, 50 pts, due: 23:59 pm, May 22th, 2022
    
    Your name:          Student ID: 

## Instruction 

- In this assignment, you will learn how to extract Lucas-Kanade Optical Flow, how to segment a image using mean shift, and how to build a simple neural network.
    - **Problem 1: Lucas-Kanade Optical Flow (10 points)**
        - Implement the `opticalFlow` function.
    - **Problem 2: Centroid Shift (10 points)**
        - Implement the `move` function.
    - **Problem 3: Mean Shift (10 points)**
        - Implement the `mean_shift` function.
    - **Problem 5: Build a Neural Network (10 points)**
        - Implement the `build_model` function.
    - **Problem 5: Train a Neural Network (10 points)**
        - Implement the `train_model` function.

- Your job is to implement the sections marked with TODO to complete the tasks.

- Submit your assignments onto **Canvas** by the due date. Upload a <code>zip</code> file containing:

    (1) The saved/latest <code>.ipynb</code> file including the output of all cells.
    
## Note
- This is an **individual** assignment. All help from others (from the web, books other than text, or people other than the TA or instructor) must be clearly acknowledged. 
- Don't use any magic function from other libraries. You will get **no credit** if a Gaussian function from a known library is used when you are asked to implement “Gaussian Filtering” from scratch.
- Don't change the input and output structure of pre-defined functions. Most coding parts can be finished with less than 5 lines of codes.
- Make sure you have installed required packages: <code>numpy</code>, <code>matplotlib</code>, <code>PIL</code>, <code>opencv</code>, <code>scipy</code>, <code>scikit-learn</code>, <code>scikit-image</code>, <code>tensorflow</code>, <code>h5py</code>. The code is tested with <code>tensorflow==2.8.0</code> and <code>h5py==3.6.0</code>, but it is possible that this code can run with other versions. 

## Objective 

- **Task 1:** Lucas-Kanade Optical Flow
- **Task 2:** Mean Shift Image Segmentation
- **Task 3:** Neural Network

Load the Dependencies
-----

In [ ]:
import cv2
import numpy as np
from skimage import io
import requests
from PIL import Image
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
from collections import OrderedDict
import pandas as pd
import scipy.io as sio
import sys
import math

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from tensorflow import keras
from scipy.io import loadmat
import numpy as np     
import h5py                                       

Problem 1: Lucas-Kanade Optical Flow (10 points)
-----

> Lucas-Kanade method is basically an estimate of the movement of interesting
features in successive images of a scene. 

> Lucas-Kanade assumes that (a) the intensity of the pixel does not change when it moves from frame1 to frame2 (b) small motion (c) motion field and optical flow is constant within a small neighborhood.

Implement the **opticalFlow** function below.


In [ ]:
def inRange(coordinates, limits):
    ''' 
    inRange checks whether the given cordinates line in the given image limits
    cordinates, limits are tuples i.e., (X,Y) 
    '''
    x,y = coordinates
    X_Limit, Y_Limit = limits
    return 0 <= x and x < X_Limit and 0 <= y and y < Y_Limit

In [ ]:
def opticalFlow(old_frame, new_frame, window_size, min_quality=0.01):
    '''
    OpticalFlow calculates the displacements in X and Y directions i.e., (u,v)
    given two consecutive images varying with time

    Input:
        old_frame: Image from time T. shape (m,n)
        new_frame: Image from time T+1. shape (m,n)
        window_size: the window size in which the Lucas-Kanade assumption holds. int
        min_quality; controls what features we want to track. int
    Output:
        dst_img: destination image, shape (m, n, 3)
    
    TODO: Implement the computation of u,v 
    '''
    max_corners = 10000
    min_distance = 0.1
    # Corners are better points to be tracked. 
    # Here we call OpenCV implemented Shi-Tomasi algorithm to find some corner points
    feature_list = cv2.goodFeaturesToTrack(old_frame, max_corners, min_quality, min_distance)

    w = int(window_size/2)

    old_frame = old_frame / 255
    new_frame = new_frame / 255

    #Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1, 1], [-1, 1]])
    kernel_y = np.array([[-1, -1], [1, 1]])
    kernel_t = np.array([[1, 1], [1, 1]])

    fx = cv2.filter2D(old_frame, -1, kernel_x)              #Gradient over X
    fy = cv2.filter2D(old_frame, -1, kernel_y)              #Gradient over Y
    ft = cv2.filter2D(new_frame, -1, kernel_t) - cv2.filter2D(old_frame, -1, kernel_t)  #Gradient over Time


    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:        #   for every corner
            j, i = feature.ravel()		#   get cordinates of the corners (i,j). They are stored in the order j, i
            i, j = int(i), int(j)		#   i,j are floats initially

            I_x = fx[i-w:i+w+1, j-w:j+w+1].flatten()
            I_y = fy[i-w:i+w+1, j-w:j+w+1].flatten()
            I_t = ft[i-w:i+w+1, j-w:j+w+1].flatten()

            b = np.reshape(I_t, (I_t.shape[0],1))
            A = np.vstack((I_x, I_y)).T

            ################## Your code begins here #####################
            Pinv_A = 
            U = # Solving for (u,v) i.e., U here
            ################## Your code ends here #####################

            u[i,j] = U[0][0]
            v[i,j] = U[1][0]

    return (u,v)


def drawSeperately(old_frame, new_frame, U, V):
    '''
    Create a plot of the displacement vectors given (u,v) and plot the two images and displacement in a row.
    '''
    displacement = np.ones_like(img2)            #Fill the displacement plot with White background
    displacement.fill(255.)
    line_color =  (0, 0, 0)
    # draw the displacement vectors
    for i in range(img2.shape[0]):
        for j in range(img2.shape[1]):
            start_pixel = (i,j)
            end_pixel = ( int(i+U[i][j]), int(j+V[i][j]) )
            #check if there is displacement for the corner and endpoint is in range
            if U[i][j] and V[i][j] and inRange( end_pixel, img1.shape ):     
                displacement = cv2.arrowedLine( displacement, start_pixel, end_pixel, line_color, thickness =2)

    figure, axes = plt.subplots(1,3)
    axes[0].imshow(old_frame, cmap = "gray")
    axes[0].set_title("first image")
    axes[1].imshow(new_frame, cmap = "gray")
    axes[1].set_title("second image")
    axes[2].imshow(displacement, cmap = "gray")
    axes[2].set_title("displacements")
    figure.tight_layout()
    plt.show()

In [ ]:
img1_url = "https://raw.githubusercontent.com/Utkal97/Object-Tracking/main/Inputs/basketball1.png"
img2_url = "https://raw.githubusercontent.com/Utkal97/Object-Tracking/main/Inputs/basketball2.png"

img1 = Image.open(requests.get(img1_url, stream=True).raw)
img1 = np.array(img1)
print(f'image1 shape: {img1.shape}')
img2 = Image.open(requests.get(img2_url, stream=True).raw)
img2 = np.array(img2)
print(f'image2 shape: {img2.shape}')
plt.subplot(121),plt.imshow(img1, cmap="gray"),plt.title('Image1')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img2, cmap="gray"),plt.title('Image2')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# Obtain (u,v) from Lucas Kanade's optical flow approach
U, V = opticalFlow(img1, img2, 3, 0.05)

# Save results
drawSeperately(img1, img2, U, V)

Mean Shift Setup
-----

Before we start image segmentation with mean shift aigorithm, first we need to define some global variables and utility functions.

In [ ]:
# global variables
MIN_DISTANCE = 5
GROUP_DISTANCE = 10
COLOR_POOL = [[1,50,200],[150,200,9],[255,0,0],[10,79,0],\
			  [200,100,0],[25,38,0],[78,250,250],[158,143,120],[28,222,98],\
			  [255,250,199],[100,150,100],[200,50,1],[250,250,78],[15,38,245],\
			  [123,234,90]]

In [ ]:
def euclidean_distance(x1, x2):
	# weighted on the distance more
	dis = (sum((x1[:2] - x2[:2])**2))*1.5
	color = sum((x1[2:] - x2[2:])**2)
	return np.sqrt(dis + color)

# formula: (1/(sigma*sqrt(2*pi)))*e^(-x^2/(2*sigma^2))
def gaussian(distance, sigma):
	return (1/(sigma*math.sqrt(2*math.pi)))*np.exp(-0.5*((distance/sigma))**2)
 
def get_neighbors(all_points, center_point, sigma):
	neighbors = []
	for p in all_points:
		distance = euclidean_distance(p, center_point)
		if distance <= sigma:
			neighbors.append(p)
	return neighbors

def find_min_dis(group, pos):
	min_distance = sys.float_info.max
	for p in group:
		min_distance = min(min_distance, euclidean_distance(p, pos));
	return min_distance

Problem 2: Centroid Shift (10 points)
=========

You should finish the **move** function. It receives a center point, points within a certain window of center point, and standard deviation of the gaussian function for weighted averaging. It computes a new centroid by using weighted average of all the points within a centain window.


In [ ]:

def move(center_point, points, sigma):
	'''
    Computes the weighted mean of the density in the window determined by the Gaussian kernel
	(sum of gaussian(distance)*point) / (sum of gaussian(distance))

    Input:
        center_point: The center of the window. shape(5, ). contains x,y,r,g,b
        points: A list of points in the window. each element if of shape(5, ), containing contains x,y,r,g,b
        sigma: The standard deviation of gaussian function. float
    Output:
        new_center_point: The new center of the window. shape(5, ). contains x,y,r,g,b
    
    TODO: Implement the move function
    '''
	sum_x = 0
	sum_y = 0
	sum_r = 0
	sum_g = 0
	sum_b = 0
	sum_weight = 0

	for p in points:
		distance = euclidean_distance(p, center_point)
		weight = gaussian(distance, sigma)
        ################## Your code begins here #####################
		sum_x +=
		sum_y +=
		sum_r +=
		sum_g +=
		sum_b +=
		sum_weight +=
        ################## Your code ends here #####################

	return np.array([(sum_x / sum_weight), (sum_y / sum_weight), \
		(sum_r / sum_weight), (sum_g / sum_weight), (sum_b / sum_weight)])

Group Results
-----

There is no guarantee that two different points will eventually converge to exactly the same mode. In fact, in practice that barely happens. Thus, two converged centroids that are close enough are classified as belonging to the same cluster. 

In [ ]:
def group_clusters(positions, points):
	# label each original points
	labels = [0]
	# a list to group the positions, for the purpose of calculating the min distance
	grouped_positions = [[positions[0]]]
	# a list to group the original points
	# once we know the lable of the original point, we can quickly find other elements in the same group
	grouped_points = [np.array([points[0]])]
	total_index = 1
	for i in range(1, len(positions)):
		need_new_index = True
		for index, group in enumerate(grouped_positions):
			if find_min_dis(group, positions[i]) <= GROUP_DISTANCE:
				labels.append(index)
				grouped_positions[index].append(positions[i])
				grouped_points[index] = np.vstack([grouped_points[index], points[i]])
				need_new_index = False
				break
		if need_new_index:
			labels.append(total_index)
			grouped_positions.append([positions[i]])
			grouped_points.append(np.array([points[i]]))
			total_index += 1

	return labels, np.array(grouped_points)

Problem 3: Mean Shift (10 points)
=========

You should finish the **mean_shift** function. It implements the main logic of mean shift algorithm.


In [ ]:
def mean_shift(points, sigma):
	'''
    Mean shift main function. Calls previous functions to finish mean shift segmentation.

    Input:
        points: shape (m, 5). each row is the concatenation of pixel coordinates x,y and pixel values r,g,b
        sigma: The standard deviation of gaussian function. float
    Output:
        labels: a list of indices of cluster to which each pixel belongs to
		grouped_points: shape (g, n, 5), where g is the number of clusters, 
		n is the pixel numbers in each cluster, 5 gor x,y,r,g,b
    
    TODO: Implement the mean_shift function
    '''
	# we mark a point as visited when it is converged(measured by distance)
	# type: {tuple: boolean}
	visited = OrderedDict((tuple(p), False) for p in points)

	# mapping from the original points to points after shifted
	# type: {tuple: ndarray}
	original_to_shifted = OrderedDict((tuple(p), p) for p in points)

	converged = False
	iteration = 0																										

	# if not all points have converged
	while not converged or iteration > 30:
		converged = True
		for original_point in original_to_shifted:
			cur_pos = original_to_shifted[original_point]
			if visited[tuple(cur_pos)]:
				continue
            ################## Your code begins here #####################
			neighbors = 
			next_pos = 
			original_to_shifted[original_point] = 
			################## Your code ends here #####################
			distance_dif = euclidean_distance(cur_pos, next_pos)
			if distance_dif <= MIN_DISTANCE:
				visited[tuple(next_pos)] = True
			else:
				visited[tuple(next_pos)] = False
				converged = False
		iteration += 1

	positions = list(original_to_shifted.values())

	# after all ponits are converge d, we group and label the clusters
	labels, grouped_points = group_clusters(positions, points)

	return labels, grouped_points

In [ ]:
def segmentation(image, points, width, height):
    labels, shifted_points = mean_shift(points, 5)

    new_image = np.zeros_like(image)
    # draw the result of the segmentation
    for pts in shifted_points:
        for point in pts:
            x = point[1]
            y = point[0]
            new_image[y, x] = np.mean(pts, axis=0).astype(np.uint8)[2:]

    new_image_resized = cv2.resize(new_image,(width,height),interpolation=cv2.INTER_CUBIC)
    return new_image_resized

In [ ]:
url = "https://news.ucsc.edu/2011/06/images/slugcloseup350.jpg"  

image = io.imread(url)
original_height = int(image.shape[0])
original_width = int(image.shape[1])
print(f'image shape: {image.shape}')
plt.xticks([]), plt.yticks([])
plt.imshow(image)

In [ ]:
# resize to a smaller size to reduce computational cost
# Generally, the larger the resolution, the better the result, and the longer it takes
image = cv2.resize(image,(int(original_height/5),int(original_width/5)),interpolation=cv2.INTER_CUBIC)
all_points = []

for i in range(0, image.shape[0]):
  for j in range(0, image.shape[1]):
    all_points.append(np.array([i,j] + list(image[i,j]))) # revised

segmented_image = segmentation(image, all_points, original_width, original_height)
plt.imshow(segmented_image),plt.title('nSegmentation Result')
plt.xticks([]), plt.yticks([])
plt.show()

Neural Network Setup
-----

Before we start building a neural network, first we need to load the dataset and split it into train/val/test set. Here we use Street View House Numbers (SVHN) dataset, which is a digit classification benchmark dataset that contains 600,000 32×32 RGB images of printed digits (from 0 to 9) cropped from pictures of house number plates. 

In [ ]:
val_size = 10000
# download SVHN_train.hdf5 and SVHN_test.hdf5 from https://drive.google.com/file/d/17Ros6RTqPUN_SCEmJT_eiuLx3OOpbPOU/view?usp=sharing, https://drive.google.com/file/d/1VHMJk0PfJnVsvgCqoV_HhSGD9E7iM8QE/view?usp=sharing
with h5py.File('SVHN_train.hdf5', 'r') as f:
    shape = f["X"].shape
    x_train = f["X"][:shape[0]-val_size]
    y_train = f["Y"][:shape[0]-val_size].flatten()
    x_val = f["X"][shape[0]-val_size:]
    y_val = f["Y"][shape[0] - val_size:].flatten()

with h5py.File('SVHN_test.hdf5', 'r') as f:
    x_test = f["X"][:]
    y_test = f["Y"][:].flatten()

y_train = keras.utils.to_categorical(y_train, 10)
y_val = keras.utils.to_categorical(y_val, 10)
y_test = keras.utils.to_categorical(y_test, 10)


Problem 4: Build a Neural Network (10 points)
=========

Build your convolutional neural networks by adding some layers. You should use 4 convolution layers and ReLU as the default activation function. The kernel size of both layers should be 3x3. Use 32 and 64 as the number of filters for the first and the second convolutional layers, respectively. Use 128 as the number of filters for two last convolution layers, and 3x3 as the kernel size.After that, flatten your input and add two more dense layers. There should be 1024 units in the first dense with ReLU activation, and use 10 hidden units in the second dense layer with softmax activation. 



In [ ]:

def build_model():
    '''
    add multiple layers as instructed to the model.

    Output:
        a keras neural network model with certain layers.
    
    TODO: Implement the build_model function
    '''
    model = tf.keras.models.Sequential()

    ################## Your code begins here #####################
    # Add layers
    model.add() # call model.add multiplpe times until all necessary layers are added
    ################## Your code ends here #####################

    return model


Problem 5: Train a Neural Network (10 points)
=========

Compile model here and set your initial hyperparameters. Use SGD as the optimizer with initial learning rate 0.01 and the momentum = 0.9. You could choose 'categorical_crossentropy' as your loss function, and the metrics should be 'accuracy'. After that, train your model for 10 epochs. 

In [ ]:
def train_model(model):
    '''
    Define the optimizer, compile model, and train the model.
    
    TODO: Implement the train_model function
    '''
    ################## Your code ends here #####################
    sgd = 
    # compile before start training. you need to specify parameters.
    model.compile()
    # use model.fit for training. you need to specify parameters.
    model.fit()
    ################## Your code ends here #####################

In [ ]:
model = build_model()
train_model(model)
test_loss, test_acc = model.evaluate(x_test, y_test)
print(test_loss, test_acc)